<a href="https://colab.research.google.com/github/JackBlake-zkq/robust-edge-inference/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from fa_ensemble import FiniteAggregationEnsemble
from torchvision.datasets import CIFAR10
from torchvision import transforms
import torch
from torch import nn, optim
import random
import numpy
from tqdm import tqdm
import ssl
from models.resnet import ResNet18, ResNetSmall

In [2]:
ssl._create_default_https_context = ssl._create_unverified_context
trainset = CIFAR10(root='./datasets/CIFAR10', train=True, download=True, transform=transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
]))
testset = CIFAR10(root='./datasets/CIFAR10', train=False, download=True, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
]))

Files already downloaded and verified
Files already downloaded and verified


In [3]:
def train_base_model(partition_number, train_subset):
    seed = partition_number
    random.seed(seed)
    numpy.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    curr_lr = 0.01
    epochs = 100
    device = (
        "cuda"
        if torch.cuda.is_available()
        else "mps"
        if torch.backends.mps.is_available()
        else "cpu"
    )
    device += ":" + str(partition_number)

    trainloader = torch.utils.data.DataLoader(train_subset, batch_size=128, shuffle=True, num_workers=1)
    print("subset has ", len(train_subset), "data points")
    
    ssl._create_default_https_context = ssl._create_unverified_context
    net = ResNet18()

    net = net.to(device)

    criterion = nn.CrossEntropyLoss()

    optimizer = optim.SGD(net.parameters(), lr=curr_lr, momentum=0.9, weight_decay=0.0005, nesterov= True)

    # Training
    net.train()
    for epoch in tqdm(range(epochs)):
        for (inputs, targets) in trainloader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
        if (epoch in [60,120,160]):
            curr_lr = curr_lr * 0.2
            for param_group in optimizer.param_groups:
                param_group['lr'] = curr_lr


    net.eval()
    nomtestloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=1)
    correct = 0
    total = 0
    for (inputs, targets) in nomtestloader:
        inputs, targets = inputs.to(device), targets.to(device)
        with torch.no_grad():
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            _, predicted = outputs.max(1)
            correct += predicted.eq(targets).sum().item()
            total += targets.size(0)
        break
    acc = 100.*correct/total
    print(f'Estimated accuracy for base model {partition_number}: {str(acc)}%')

    return net


In [4]:
ensemble = FiniteAggregationEnsemble(trainset, testset, train_base_model, 10, k=50, d=1, state_dir="cifar10_k50")
for i in range(50):
    ensemble.train_base_model(i)

Base model 0 already exists
Base model 1 already exists
Base model 2 already exists
Base model 3 already exists
Base model 4 already exists
Base model 5 already exists
Base model 6 already exists
Base model 7 already exists
Base model 8 already exists
Base model 9 already exists
Base model 10 already exists
Base model 11 already exists
Base model 12 already exists
Base model 13 already exists
Base model 14 already exists
Base model 15 already exists
Base model 16 already exists
Base model 17 already exists
Base model 18 already exists
Base model 19 already exists
Base model 20 already exists
Base model 21 already exists
Base model 22 already exists
Base model 23 already exists
Base model 24 already exists
Base model 25 already exists
Base model 26 already exists
Base model 27 already exists
Base model 28 already exists
Base model 29 already exists
Base model 30 already exists
Base model 31 already exists
Base model 32 already exists
Base model 33 already exists
Base model 34 already ex

In [5]:
ensemble.eval("softmax_median")

testset predictions already computed, using those...
Certificates already computed, using those...
Base classifier accuracy: 50.0012
Ensembe Accuracy: 65.60000000000001%
Certified Radius (for at least half of inputs): 6


In [6]:
ensemble.eval("logit_median")

testset predictions already computed, using those...
Certificates already computed, using those...
Base classifier accuracy: 50.0012
Ensembe Accuracy: 65.49000000000001%
Certified Radius (for at least half of inputs): 5


In [7]:
ensemble.eval(mode="label_voting")

testset predictions already computed, using those...
Certificates already computed, using those...
Base classifier accuracy: 50.0012
Ensembe Accuracy: 65.36%
Certified Radius (for at least half of inputs): 6


In [8]:
ensemble.distill(ResNetSmall(), 'softmax_median', lr=1e-3, epochs=10)

trainset predictions already computed, using those...


/Users/jackblake/Documents/UW_Courswork/robust-edge-inference/training/fa_ensemble.py:334: UserWarning: torch.sort is supported by MPS on MacOS 13+, please upgrade. Falling back to CPU (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/Sort.mm:45.)
  softmaxes, _ = softmaxes_by_class.to(device).sort(dim=2)


Epoch 1/10


100%|██████████| 391/391 [01:35<00:00,  4.10it/s]


Trainset Accuracy: 35.354%
Epoch 2/10


...

Trainset Accuracy: 61.836%
Epoch 10/10


100%|██████████| 391/391 [01:34<00:00,  4.14it/s]


Trainset Accuracy: 62.598%
Finished training student, saving to ensembles/cifar10_k50/student_softmax_median.pkl
Evaluating Student


100%|██████████| 79/79 [00:10<00:00,  7.33it/s]

Accuracy for student: 62.53%


In [9]:
ensemble.distill(ResNetSmall(), mode='logit_median', lr=1e-3, epochs=10)

trainset predictions already computed, using those...
Epoch 1/10


100%|██████████| 391/391 [01:34<00:00,  4.13it/s]


Trainset Accuracy: 46.488%
Epoch 2/10


100%|██████████| 391/391 [01:35<00:00,  4.11it/s]


Trainset Accuracy: 59.46%
Epoch 3/10


...

Trainset Accuracy: 64.19200000000001%
Epoch 10/10


100%|██████████| 391/391 [01:35<00:00,  4.08it/s]


Trainset Accuracy: 64.344%
Finished training student, saving to ensembles/cifar10_k50/student_logit_median.pkl
Evaluating Student


100%|██████████| 79/79 [00:10<00:00,  7.32it/s]

Accuracy for student: 64.03%


In [10]:
ensemble.distill(ResNetSmall(), 'label_voting', lr=1e-3, epochs=10)

trainset predictions already computed, using those...
Epoch 1/10


100%|██████████| 391/391 [01:36<00:00,  4.05it/s]


Trainset Accuracy: 38.751999999999995%
Epoch 2/10


100%|██████████| 391/391 [01:36<00:00,  4.04it/s]


Trainset Accuracy: 52.308%
Epoch 3/10


...

Trainset Accuracy: 63.212%
Epoch 10/10


100%|██████████| 391/391 [01:34<00:00,  4.14it/s]


Trainset Accuracy: 63.576%
Finished training student, saving to ensembles/cifar10_k50/student_label_voting.pkl
Evaluating Student


100%|██████████| 79/79 [00:10<00:00,  7.43it/s]

Accuracy for student: 63.43%
